# Capstone Project Title
___
**Author**: Evan Holder

## Table of Contents
1. [Background](#Background)<br>
2. [Business Problem](#Business-Problem)<br>
3. [Data Collection](#Data-Collection)<br>
4. [Data Cleaning](#Data-Cleaning)<br>
5. [Data Preparation](#Data-Preparation)<br>
    5.1 [Encode Categorical Features](#Encode-Categorical-Features)<br>
    5.2 [Scaling Continuous Features](#Scaling-Continuous-Features)<br>
    5.3 [Multicolinearity](#Multicolinearity)<br>
6. [Modeling the Actual Price](#Modeling-price_actual)<br>
    6.1 [Lasso Regression](#Lasso-Regression)<br>
    6.2 [XGBoost Regression](#XGBoost-Regression)<br>
    6.3 [Neural Networks](#Neural-Networks)<br>
    6.3.1 [Neural Network 1-to-1](#Neural-Network-1-to-1)<br>
    6.3.2 [Neural Network 24-to-24](#Neural-Network-24-to-24)<br>
    6.3.3 [LSTM Neural Network](#Neural-Network-LSTM)<br>
    6.3.4 [DNN-LSTM Neural Network](#Neural-Network-DNN-LSTM)<br>
7.0 [Modeling the Price Residual](#Modeling-Price_Residual)<br>
8.0 [Impact of Weather Features on Price](#Weather)<br>

### Background <a class="anchor" id="Background"></a>
___

### Business Problem <a class="anchor" id="Business-Problem"></a>
___

### Data Collection <a class="anchor" id="Data-Collection"></a>
___


### Data Cleaning <a class="anchor" id="Data-Cleaning"></a>
____

### Import Libraries and Functions  <a class="anchor" id="Import-Libraries-and-Funtions"></a>
___
Data manipulation, cleaning, massaging: pandas, numpy<br>
Modeling: sklearn, keras<br>
Plotting: matplotlib<br>
Custom functions: function.py

In [1]:
# Libraries for data cleaning, massaging:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import datetime as dt


# Modeling Libraries
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso

import tensorflow as tf
from tensorflow import keras
from keras import layers, models, regularizers
from tensorflow.keras.layers import TimeDistributed
from sklearn.preprocessing import MinMaxScaler

# Save Models
import pickle

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Manipulate directories 
import os

# Import custom functions
os.chdir('../scripts')
from functions import split_data, sMAPE, SMAPE, compute_metrics, r2,impute_immediate_mean
from functions import resample, plot_metric_range, compile_fit, ensemble_nn
os.chdir('../notebooks')

### Import Data <a class="anchor" id="Import-Data"></a>
___

In [2]:
# Read in data
df_lag = pd.read_csv('../data/clean/df_clean_lag.csv', index_col=0, parse_dates=True)

### Data Preparation <a class="anchor" id="Data-Preparation"></a>
___
In this project, we'll focus on three main algorithms types: Lasso Regression, XGBoost, and Neural Networks.  We'll need to prepare the data in slightly different ways for each of the these model types. Much of the preprocessing was already taken care of as part of the steps list above in [Data Cleaning](#Data-Cleaning). The remaining steps are model specific, and so are prepared below:<br><br>
**Lasso Regression**:<br>
* Encode the categorical features
* Remove mulitcolinearities

**Neural Networks**:<br>
* Encode the categorical features

**XGBoost**: All preprocessing steps were previously 
While not required, we'll scale the continuous features for neural networks. And finally for lasso regression, it would be wise to remove any multicolinearities in the data. In order to run each of these models, I'll copy the dataset and process

### Encode Catergorical Features <a class="anchor" id="Encode-Categorical-Features"></a>

In [3]:
# Get Categorical columns
categorical = df_lag.select_dtypes(include='object')

# Instationate wind_dir_coder LabelEncoder, fit
wind_dir_coder = LabelEncoder()
wind_dir_coder.fit(df_lag['wind_madrid_lag'])

# Transform wind_direction cols
for col in categorical.filter(regex='wind').columns:
    df_lag[col] = wind_dir_coder.transform(df_lag[col])
    

# Stack condition columns into single col
stacked_conditions = categorical.filter(regex='condition').stack()

# Instantiate condition_coder LabelEncoder, fit on stacked conditions
condition_coder = LabelEncoder()
condition_coder.fit(stacked_conditions)

# Transform condition cols
for col in categorical.filter(regex='condition').columns:
    df_lag[col] = condition_coder.transform(df_lag[col])

In [4]:
# Get price components not to be used in modeling
price_cols = df_lag.filter(regex='price').columns.to_list()[1:]
price_cols.remove('price_day_ahead')


### Scaling Continuous Features <a class="anchor" id="Scaling-Continuous-Features"></a>
For neural networks, continuous features do not necessarily need to be scaled. However according to this [article](https://www.sciencedirect.com/science/article/pii/S030626191830196X#s0235), which uses neural networks to predict electrical prices, scaling your continuous features generally increases accuracy of deep learning models on the validation set.  We'll give it a go here and scale the the continuos features between [-1,1] for the neural networks we'll train later.

In [5]:
# Copy the dataframe for neural networks
df_nn = df_lag.drop(columns=price_cols).copy()
continuous = df_nn.select_dtypes(exclude='object').filter(regex='^(?!.*price).*').columns

# Get rid of negatives
time = dt.datetime(2021,3,24,22)
df_nn.loc[time, 'dew_point_bilbao_lag'] = impute_immediate_mean(df_nn['dew_point_bilbao_lag'], time)

# Rescale data [-1,1]
scaler = MinMaxScaler(feature_range=(-1, 1))
df_nn[continuous] = scaler.fit_transform(df_nn[continuous])

### Multicolinearity <a class="anchor" id="Multicolinearity"></a>
One of the assumptions for regression is that featu
res do not contain multicolinearities.  In this section, we'll investigate the predictors and eliminate any multicolinearities in preparation for a lasso regression. We'll need to find out which features are correlated with each other, and remove some of them to rid our dataset of multicolinearities. The steps are outlined below:
1. Copy the dataframe, we'll modify this dataset for use in lasso regression
2. Get the correlations between predictors, sort them in descending order
3. Get the correlations between each individual predictor and the response variable
4. Get the features which have a correlation greater than 0.8, add the feature which correlates less with price_actual to the drop list
5. Drop the features in the drop list

In [6]:
# Copy dataset for lasso regression specific preparation
df_lr = df_lag.drop(columns=price_cols).copy()

# Create correlation matrix predictors to predictors
corr = df_lr.drop(columns='price_actual').corr().abs().stack().reset_index().sort_values(0, ascending=False)
corr.rename(columns={0:'cor'}, inplace=True)  # Rename correlation column
corr = corr.loc[corr['cor']!=1]  # remove correlations between same variables
corr.drop_duplicates(subset='cor', inplace=True) # remove duplicate correlations
corr.reset_index(drop=True, inplace=True) # Reset the index
corr.cor =corr.cor.apply(lambda x: round(x,3))  # Round

# Create correlation matrix predictors to response variable
corr_price = df_lr.corr()['price_actual'].reset_index().sort_values('price_actual', ascending=False)
corr_price = corr_price.loc[corr_price['price_actual']!=1] # remove correlations between same variables
corr_price.reset_index(drop=True, inplace=True)  # Reset the index


drop = []

# For each feature pair where corr > 0.8, add feature with lower corr to price_actual to drop list
for row in range(len(corr.loc[corr.cor>.8])):
    var1 = corr.loc[row,'level_0'] # Get var1 name
    var2 = corr.loc[row,'level_1'] # Get var2 name
    var1_corr = float(corr_price.loc[corr_price['index'] == var1, 'price_actual'])  # Get var1 corr
    var2_corr = float(corr_price.loc[corr_price['index'] == var2, 'price_actual'])  # Get var2 corr
    
    # Add the lower correlation to the drop list
    if var1_corr > var2_corr:
        drop.append(var2)
    else:
        drop.append(var1)
        
# Drop the features in the drop listi
df_lr.drop(columns=drop, inplace=True)

In [7]:
df_xg = df_lag.drop(columns=price_cols)

### Modeling `price_actual` <a class="anchor" id="Modeling-price_actual"></a>
___
WRITE SOMETHING
Create results_actual dataframe to hold results

Add TSO (price_day_ahead) as benchmark prediction to beat

In [8]:
# Benchmark results
TSO_train = df_lag.loc[:'2019', 'price_day_ahead']
TSO_val = df_lag.loc['2020', 'price_day_ahead']

actual_train = df_lag.loc[:'2019', 'price_actual']
actual_val = df_lag.loc['2020', 'price_actual']

# Create dataframe
results_actual = pd.DataFrame(index=['Parameters','SMAPE_train', 'SMAPE_val', 'r2_train', 'r2_val'])

# Add the baseline TSO predictions
results_actual['TSO'] = ['None',
                                    round(sMAPE(actual_train, TSO_train), 3),
                                    round(sMAPE(actual_val, TSO_val), 3),
                                    round(r2(actual_train, TSO_train), 3), 
                                    round(r2(actual_val, TSO_val),3)]
results_actual

,TSO
Parameters,None
SMAPE_train,16.03
SMAPE_val,16.922
r2_train,0.954
r2_val,0.971


In [9]:
X_train, y_train, X_val, y_val = split_data(df_lr, 2020, 'price_actual')
with open('../models/Lasso.pickle', 'rb') as file:
    lasso = pickle.load(file)
with open('../models/Lasso1.pickle', 'rb') as file:
    lasso1 = pickle.load(file)
with open('../models/Lasso2.pickle', 'rb') as file:
    lasso2 = pickle.load(file)
with open('../models/XGBoost.pickle', 'rb') as file:
    xg = pickle.load(file)
with open('../models/XGBoost1.pickle', 'rb') as file:
    xg1 = pickle.load(file)
with open('../models/XGBoost2.pickle', 'rb') as file:
    xg2 = pickle.load(file)
with open('../models/XGBoost3.pickle', 'rb') as file:
    xg3 = pickle.load(file)
nn1 = keras.models.load_model('../models/nn1', custom_objects={'SMAPE':SMAPE})
nn2 = keras.models.load_model('../models/nn2', custom_objects={'SMAPE':SMAPE})
nn3 = keras.models.load_model('../models/nn3', custom_objects={'SMAPE':SMAPE})
nn4 = keras.models.load_model('../models/nn4', custom_objects={'SMAPE':SMAPE})
nn5 = keras.models.load_model('../models/nn5', custom_objects={'SMAPE':SMAPE})
nn6 = keras.models.load_model('../models/nn6', custom_objects={'SMAPE':SMAPE})
    

results_actual['Lasso'] = compute_metrics(lasso, 'Vanilla', (X_train, y_train), (X_val, y_val))
results_actual['Lasso1'] = compute_metrics(lasso, {'num_features':5}, (X_train, y_train), (X_val, y_val))
X_train1, X_val1 = X_train.drop(columns='price_day_ahead'), X_val.drop(columns='price_day_ahead')
results_actual['Lasso2'] = compute_metrics(lasso2, {'price_day_ahead':False}, (X_train1, y_train), (X_val1, y_val))
X_train, y_train, X_val, y_val = split_data(df_xg, 2020, 'price_actual')
results_actual['XGBoost'] = compute_metrics(xg, 'Vanilla',(X_train, y_train), (X_val, y_val))
results_actual['XGBoost1'] = compute_metrics(xg1, {'max_depth':2},(X_train, y_train), (X_val, y_val))
X_train1, X_val1 = X_train.drop(columns='price_day_ahead'), X_val.drop(columns='price_day_ahead')
results_actual['XGBoost2'] = compute_metrics(xg2, {'max_depth':2,
                                                   'price_day_ahead':False},(X_train1, y_train), (X_val1, y_val))
results_actual['XGBoost3'] = compute_metrics(xg3,
                                             {'max_depth':16, 'price_day_ahead':False},
                                             (X_train1, y_train), 
                                             (X_val1, y_val))


X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')
results_actual['nn1'] = compute_metrics(nn1, '1-to-1', (X_train,y_train), (X_val, y_val))

X_train, y_train, X_val, y_val = split_data(df_nn.drop(columns='price_day_ahead'), 2020, 'price_actual')
results_actual['nn2'] = compute_metrics(nn2, '1-to-1, price_day_ahead:False', (X_train,y_train), (X_val, y_val))

X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')
X_train, y_train = resample((X_train, y_train), 24, 24, 24)
X_val, y_val = resample((X_val,y_val), 24, 24, 24)
results_actual['nn3'] = compute_metrics(nn3, '24-to-24', (X_train,y_train), (X_val, y_val))


X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')
X_train, y_train = resample((X_train, y_train), 24*7, 24, 24)
X_val, y_val = resample((X_val,y_val), 24*7, 24, 24)
results_actual['nn4'] = compute_metrics(nn4, 'LSTM, 7-day input', (X_train,y_train), (X_val,y_val))
results_actual['nn5'] = compute_metrics(nn5, 'LSTM, 7-day input', (X_train,y_train), (X_val,y_val))

In [10]:
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')
X_train_lstm = X_train.filter(regex='lag')
X_train_dnn = X_train.drop(columns=X_train_lstm.columns)
X_val_lstm = X_val.filter(regex='lag')
X_val_dnn = X_val.drop(columns=X_val_lstm.columns)
X_train_dnn, y_train_dnn = resample((X_train_dnn, y_train), 24, 24, 24)
X_val_dnn, y_val_dnn = resample((X_val_dnn, y_val), 24, 24, 24)
X_train_lstm, y_train_lstm = resample((X_train_lstm, y_train), 24, 24, 24)
X_val_lstm, y_val_lstm = resample((X_val_lstm, y_val), 24, 24, 24)
results_actual['nn6'] = compute_metrics(nn6,
                                        'dnn-lstm, 1-day input',
                                        ([X_train_dnn, X_train_lstm],y_train),
                                        ([X_val_dnn, X_val_lstm],y_val))

In [11]:
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,"1-to-1, price_day_ahead:False",6.849,23.042,0.907,0.428


### Lasso Regression <a class="anchor" id="Lasso-Regression"></a>
I'll start with a simple Lasso Regression.  Lasso Regression is really just a a linear regression that introduces a penalty infront of each coefficient in the model. Lasso is well-suited for datasets with high multicolinearities since it automatically selects for one of the features in a a colinear pair. As part of the [preprocessing] for this model, I also remove features with high colinearity (>0.8). In fitting the below, I'll take the following steps:

* Split the data into training (2015-2019) and validation (2020)
* Fit a Vanilla lasso regression model with max_iter=10000 to make sure that the model converges.
* Compute the output and add it to the results table.

In [12]:
# Split the data
X_train, y_train, X_val, y_val = split_data(df_lr, 2020, 'price_actual')

# Instatiate and fit model on 
lasso = Lasso(max_iter=10000)
lasso.fit(X_train, y_train)

# Add results of vanilla lasso to dataframe
results_actual['Lasso'] = compute_metrics(lasso, 'Vanilla', (X_train, y_train), (X_val, y_val))
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,24-to-24,4.022,3.848,0.983,0.979


In [33]:
with open('../models/lasso_vanilla.pickle', 'rb') as file:
    lasso_test = pickle.load(file)

The vanilla model performed extremely well.  That's great, but it also isn't suprising given that we used the `price_day_ahead` as a predictor which has an r-squared value of 0.971 on the validation set.  The model outperformed the TSO predictions in SMAPE and even increased r-squared by a small margin. Increased r-squared must mean that some of the other features were important in our prediction of `price_actual`.  I plotted the coefficients for this vanilla model on the below barchart.

<img src="../images/lasso_feature_importance.png" style="width:700px;height:272px"/>

As expected, `price_day_ahead` dominates this model, though renewable generation and waste generation have an impact, negatively affecting the price (renewable and waste increase, results in price decrease).  Other than that, the other features have very little influence on the final price.  

**Recursive Feature Elimination**<br>
As part of the [Lasso](https://github.com/EvanHolder/capstone/blob/main/notebooks/LassoRegression.ipynb) notebook, I ran a recursive feature elimination to see how the model performs with varying amounts of features in the model.  I started with a single feature (`price_day_ahead`), trained a model, and computed its metrics. Then I iteratively added in the next most important feature, trained the new model, and computed its metrics.  This process was repeated until all features were added back into the training set and the metrics were plotted as below.

![RFE_LassoRegression](../images/RFE_LassoRegression.png)

As shown above, when trained on top five features, the model minimizes r-squared.  Below, I'll train the model on these top five features (`price_day_ahead`, `renewable_lag`,`waste_lag`,`oil_lag`,`humidities_seville_lag`).

In [31]:
# Important Features
train_cols = ['humidities_bilbao_lag', 'oil_lag', 'renewable_lag', 'waste_lag', 'price_day_ahead']

# Instatiate and fit model on 
lasso1 = Lasso(max_iter=10000)
lasso1.fit(X_train[train_cols], y_train)

# Add results of vanilla lasso to dataframe
results_actual['Lasso1'] = compute_metrics(lasso1, {'num_features':5}, (X_train[train_cols], y_train), (X_val[train_cols], y_val))

# Save model
with open('../models/Lasso1.pickle', 'wb') as f:
    pickle.dump(lasso1, f)
    
# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},4.134,8.354,0.954,0.971
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn_1-1,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 1}",200.0,200.0,NaN,NaN
nn-24-24,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 162, '...",3.871,3.738,0.981,0.978


Since Lasso1 includes only five predictors, it's not surprising that the model performance (SMAPE & r-squared) decreased from the vanilla model.  Performance decreased only marginally though.  Next, let's see just how well we can do without using `price_day_ahead`.

In [11]:
# Drop 'price_day_ahead'
X_train1, X_val1 = X_train.drop(columns='price_day_ahead'), X_val.drop(columns='price_day_ahead')

# Instatiate and fit model on 
lasso2 = Lasso(max_iter=10000)
lasso2.fit(X_train1, y_train)

# Add results of vanilla lasso to dataframe
results_actual['Lasso2'] = compute_metrics(lasso2, {'price_day_ahead':False}, (X_train1, y_train), (X_val1, y_val))

# Save Model
with open('../models/Lasso2.pickle', 'wb') as f:
    pickle.dump(lasso2, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557


Lasso2 performed approximately half as well as the other two Lasso models (SMAPE, r-squared).  The difference in performance between these models indicates that we really need `price_day_ahead` to match the TSO's performance.

### XGBoost Regression <a class="anchor" id="XGBoost-Regression"></a>
XGBoost is the next choice in Machine Learning algorithms because of it's ability to learn non-linear decision boundaries.  So while we were unsuccessful in modeling without `price_day_ahead` with Lasso Regression, we'll give it another shot here with XGBoost.  XGBoost was chosen over Random Forest and other gradient boosted ensembles because it trains the fastest and generally performs better. The other great thing about XGBoost is it requires very little prepocessing. In fitting the below, I'll take the following steps:

* Split the data into training (2015-2019) and validation (2020)
* Fit a Vanilla XGBoostRegressor with random_state set to 17
* Compute the output and add it to the results table.

In [12]:
# Split the data
X_train, y_train, X_val, y_val = split_data(df_xg, 2020, 'price_actual')

# Instantiate and fit XGBRegressor
xg = XGBRegressor(random_state=17)
xg.fit(X_train, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost'] = compute_metrics(xg, 'Vanilla',(X_train, y_train), (X_val, y_val))

# Save Model
with open('../models/XGBoost.pickle', 'wb') as f:
    pickle.dump(xg, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968


The vanilla model did beat the TSO performance (SMAPE , r-squared) but surprisingly did not outperform the vanilla Lasso model. The model does not appear too overfit on r2_val, but partly on SMAPE_val.  As part of the [XGBoost notebook](https://github.com/EvanHolder/capstone/blob/main/notebooks/XGBoost.ipynb) I tried to reduce this overfitting on by iteratively training XGBoost models and tuning the below parameters.

max_depth....................... [1,2,4,6,8,10,14,16,20],<br>
gamma............................. [n/10 for n in range(11)]<br>
min_child_weight.............. [1,2,4,8,16,32],<br>
subsample........................ [n/10 for n in range(0, 12, 2)],<br>
colsample_bytree............. [n/10 for n in range(0, 12, 2)],<br>
reg_alpha......................... [.001, .01, .1, .5, 1],<br>
reg_lambda...................... [.001, .01, .1, .5, 1]

Adjusting all of these parameters in their respective ranges, there really wasn't much improvement in either of the metrics sMAPE or r2. The one exception may be `max_depth` which did see minor improvement in SMAPE when reduced set to 2.

Below, I'll run another model ith `max_depth` set to 2 and add it to the results table.

Tuning these parameters did not change the metrics in a substantial way.  `Max_depth`, the most influential, made only a marginal difference SMAPE.  

In [13]:
# Split the data
X_train, y_train, X_val, y_val = split_data(df_xg, 2020, 'price_actual')

# Instantiate and fit XGBRegressor
xg1 = XGBRegressor(random_state=17, max_depth=2)
xg1.fit(X_train, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost1'] = compute_metrics(xg1, {'max_depth':2},(X_train, y_train), (X_val, y_val))

# Save Model
with open('../models/XGBoost1.pickle', 'wb') as f:
    pickle.dump(xg1, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97


XGBoost1 does have a reduced SMAPE_val and increased r2_val, but does not outperform Lasso.  Let's next look at the feature importances from XGBoost1. I suspect that like Lasso, and given that a max tree depth of two is all we need for good performance, that `price_day_ahead` again dominates.

In [14]:
imp = pd.DataFrame({'importance':xg1.feature_importances_},
                   index=X_train.columns).sort_values(by='importance', ascending=False)
imp.head(10)

,importance
price_day_ahead,0.735356
biomass_lag,0.046353
renewable_lag,0.037915
load_forecast,0.035208
coal_lag,0.034028
waste_lag,0.020222
transmission_fs_lag,0.008342
dew_point_seville_lag,0.007706
solar_lag,0.006477
reservoir_lag,0.006204


As expected, the `price_day_ahead` has dominated.  Let's remove it and see how we do.

In [15]:
# Drop price_day_ahead
X_train1, X_val1 = X_train.drop(columns='price_day_ahead'), X_val.drop(columns='price_day_ahead')


# Instantiate and fit XGBRegressor
xg2 = XGBRegressor(random_state=17)
xg2.fit(X_train1, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost2'] = compute_metrics(xg2, {'max_depth':2,
                                                   'price_day_ahead':False},(X_train1, y_train), (X_val1, y_val))
# Save Model
with open('../models/XGBoost2.pickle', 'wb') as f:
    pickle.dump(xg2, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427


Again without our most important feature, the model performance has essentially halved (SMAPE and r2).  I further tuned XGBoost trained without `price_day_ahead` but the peformance did not improve substantially. Increasing tree depth was the only parameter that seemed to increase performance.  Below is another list of the tuned parameters, see the [notebook](https://github.com/EvanHolder/capstone/blob/main/notebooks/XGBoost.ipynb) for the plots displaying each model, and how it performed with the below list of tuned parameters.

max_depth....................... [1,2,4,6,8,10,14,16,20],<br>
gamma............................. [n/10 for n in range(11)]<br>
min_child_weight.............. [1,2,4,8,16,32],<br>
subsample........................ [n/10 for n in range(0, 12, 2)],<br>
colsample_bytree............. [n/10 for n in range(0, 12, 2)],<br>
reg_alpha......................... [.001, .01, .1, .5, 1],<br>
reg_lambda...................... [.001, .01, .1, .5, 1]

Lastly, I'll fit an XGBoost model with a max_depth of 16 and add it to the table

In [16]:
# Instantiate and fit XGBRegressor
xg3 = XGBRegressor(random_state=17, max_depth=16)
xg3.fit(X_train1, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost3'] = compute_metrics(xg3,
                                             {'max_depth':16, 'price_day_ahead':False},
                                             (X_train1, y_train), 
                                             (X_val1, y_val))
# Save Model
with open('../models/XGBoost3.pickle', 'wb') as f:
    pickle.dump(xg3, f)

#Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403


### Neural Networks <a class="anchor" id="Neural-Networks"></a>

### Neural Network (1-to-1) <a class="anchor" id="Neural-Network-1-to-1"></a>
The last class of alorithms to try and fit is neural nets.  These may fit the model better since they have the ability to find non-linear patterns through the hidden layers of the network.  In addition, they more readily take in sequence data. The intuition is that while our features may not have a direct relationship with the actual price, they may nudge the price in a certain direction over the course of time. To start off, we'll set up a simple 1-to-1 (one input, one output) model and see how that performs. The steps to fit this model are:
* Split the data into training (2015-2019) and validation (2020)
* Set the input_shape
* Establish network architecture
* Compute the output and add it to the results table.

In [13]:
# Split Data
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Define input_shape
input_shape = (X_train.shape[1],)

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(1, activation='relu'))

# Compile and Fit
nn1 = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

# Compute metrics, add to table
results_actual['nn1'] = compute_metrics(nn1, '1-to-1', (X_train,y_train), (X_val, y_val))

# Save Model
nn1.save('../models/nn1')

# Preview
results_actual.T

Epoch 1/200
1369/1369 [==============================] - 3s 2ms/step - loss: 2.0277 - SMAPE: 4.1131 - val_loss: 1.5650 - val_SMAPE: 4.6190
Epoch 2/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.4959 - SMAPE: 2.8572 - val_loss: 2.3771 - val_SMAPE: 6.7114
Epoch 3/200
1369/1369 [==============================] - 2s 2ms/step - loss: 1.3998 - SMAPE: 2.6694 - val_loss: 1.3599 - val_SMAPE: 3.9410
Epoch 4/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.3510 - SMAPE: 2.5701 - val_loss: 1.3902 - val_SMAPE: 4.0886
Epoch 5/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.3276 - SMAPE: 2.5172 - val_loss: 1.4723 - val_SMAPE: 4.3826
Epoch 6/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.2572 - SMAPE: 2.3906 - val_loss: 1.5902 - val_SMAPE: 4.7524
Epoch 7/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.2641 - SMAPE: 2.3956 - val_loss: 1.9225 - val_SMAPE: 5.6119
Epoch 8/200
1369/1369 [====

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn-24-24,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 162, '...",3.871,3.738,0.981,0.978
LSTM,"{'LSTM1': 60, 'LSTM2': 24, 'TimeDistributed': ...",8.805,10.646,0.787,0.804


The first neural network is our best yet outperforming both Lasso and XGBoost (SMAPE and r2). This is good news, and bodes well for the more complicated neural networks we'll set up soon. Before that, let's run another model without `price_day_ahead`

In [16]:
# Split Data
X_train, y_train, X_val, y_val = split_data(df_nn.drop(columns='price_day_ahead'), 2020, 'price_actual')

# Define input_shape
input_shape = (X_train.shape[1],)

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(1, activation='relu'))

# Compile and Fit
nn2 = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

# Compute metrics, add to table
results_actual['nn2'] = compute_metrics(nn2, '1-to-1, price_day_ahead:False', (X_train,y_train), (X_val, y_val))

# Save Model
nn2.save('../models/nn2')

# Preview
results_actual.T

Epoch 1/200
1369/1369 [==============================] - 3s 1ms/step - loss: 6.9831 - SMAPE: 14.1142 - val_loss: 11.3246 - val_SMAPE: 27.8058
Epoch 2/200
1369/1369 [==============================] - 2s 1ms/step - loss: 5.2084 - SMAPE: 9.8004 - val_loss: 12.2882 - val_SMAPE: 29.4160
Epoch 3/200
1369/1369 [==============================] - 2s 1ms/step - loss: 4.8063 - SMAPE: 9.0462 - val_loss: 11.5648 - val_SMAPE: 27.9300
Epoch 4/200
1369/1369 [==============================] - 2s 1ms/step - loss: 4.5451 - SMAPE: 8.5623 - val_loss: 11.2932 - val_SMAPE: 27.4478
Epoch 5/200
1369/1369 [==============================] - 2s 1ms/step - loss: 4.2693 - SMAPE: 8.0578 - val_loss: 10.4792 - val_SMAPE: 25.9620
Epoch 6/200
1369/1369 [==============================] - 2s 1ms/step - loss: 4.0567 - SMAPE: 7.6881 - val_loss: 9.6504 - val_SMAPE: 24.3069
Epoch 7/200
1369/1369 [==============================] - 2s 1ms/step - loss: 3.9240 - SMAPE: 7.4537 - val_loss: 11.2922 - val_SMAPE: 27.5251
Epoch 8/200
1

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,"1-to-1, price_day_ahead:False",6.849,23.042,0.907,0.428


As expected performance decreased (SMAPE and r2).

### Neural Network (24 to 24) <a class="anchor" id="Neural-Network-24-to-24"></a>
The next network will be a little more complicated by using sequences to predict another sequence. We'll reshape the input and output into 24 hour sequences, and 1825 batches. The network architecture change slightly too. I'll wrap the output Dense layer in a TimeDistributed wrapper.  This will transform my output into a vector of length 24, to match up with the output sequences of length 24. The steps to fit this model are the same as the 1-to-1 network with an extra step to process the input/output data.

In [14]:
results_actual.drop(columns='nn-24-24', inplace=True)

In [14]:
# Split Data
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24, 24, 24)
X_val, y_val = resample((X_val,y_val), 24, 24, 24)

# Define input_shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))

# Compile and Fit
nn3 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

# Compute metrics and add to table
results_actual['nn3'] = compute_metrics(nn3, '24-to-24', (X_train,y_train), (X_val, y_val))

# Save model
nn3.save('../models/nn3')

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,"1-to-1, price_day_ahead:False",6.849,23.042,0.907,0.428


Great, we have again improved (be it slightly) in SMAPE_val and r2_val.

### LSTM Neural Network <a class="anchor" id="LSTM-Neural-Network"></a>
Long-Short-Term-Memory neural networks are a type of recurrent neural network that have gates that allow the network to "remember" and "forget" information from a specified input window.  In this way, the model is able to better estimate time-dependent output sequences. Since much of the dataset are time-series sequences (including price_actual) it is possible that an LSTM will be able to pick up on time-dependent relationships that our other alogorithms and networks could not. The process for setting up this network is the same as the previous.  Architecture willl be change to include two LSTM layers, and a repeat vector layer. 

In [15]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24*7, 24, 24)
X_val, y_val = resample((X_val,y_val), 24*7, 24, 24)

# Input Shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(60, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

# Compile Fit
nn4 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

# Compute metrics, add to table
results_actual['nn4'] = compute_metrics(nn4, 'LSTM, 7-day input', (X_train,y_train), (X_val,y_val))

# Save Model
nn4.save('../models/nn4')

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,"1-to-1, price_day_ahead:False",6.849,23.042,0.907,0.428


While the model trained and did not overfit, it did not outpeform the previous models.  Just once, I'll try to the number of nodes in the first LSTM layer to see if that makes a difference.

In [16]:
# Input Shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

# Compile Fit
nn5 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

# Compute metrics, add to table
results_actual['nn5'] = compute_metrics(nn5, 'LSTM, 7-day input', (X_train,y_train), (X_val,y_val))

# Save Model
nn5.save('../models/nn5')

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,"1-to-1, price_day_ahead:False",6.849,23.042,0.907,0.428


Unsurprisingly, the addition of a few extra nodes in the first layer did not change the performance of the LSTM.

### DNN-LSTM Neural Network <a class="anchor" id="DNN-LSTM Neural Network"></a>
The motivation for this model comes from this [article](https://www.sciencedirect.com/science/article/pii/S030626191830196X#s0220). The idea is that time-dependent features (values from the past) can be modeled by an LSTM, but if the data represetns a specific property associated with the day ahead (forecast data) then it cannot be modeled as a time sequence.  In this case, all the columns in the dataset with `_lag` suffix are past values, while all other columns represent future predictions.  With this distinction, I'll set up a combined DNN-LSTM model:

**DNN**
* Single best non-lstm neural network
* Trained on forecast columns only

**LSTM**
* Single best lstm neural network
* Trained on the `_lag` columns

**Ensemble**<br>
In order to combine these network, I created the function [`ensemble_nn`](https://github.com/EvanHolder/capstone/blob/main/scripts/functions.py) which takes in a list of models. For each model in the list, the function changes the model's layers untrainable.  Once complete, the function sets up another mini neural network.  The architecture is the concatenated output of each model, then hidden dense layer with 24 nodes, and finally a TimeDistributed dense layer as the final output.  Once the dnn and lstm are trained on their respective columns, we can call the `ensemble_nn` function to instantiate it.  All that's left to do is fit the resulting ensemble on the data and compute the results.

In the Neural Networks [notebook](https://github.com/EvanHolder/capstone/blob/main/notebooks/NeuralNets.ipynb), I played around with both the dnn and lstm architecture input windows. In the end, this tuning did not prove to impact either the dnn or lstm significantly.

In [30]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Get the x cols for lstm network, lagged cols
X_train_lstm = X_train.filter(regex='lag')
X_train_dnn = X_train.drop(columns=X_train_lstm.columns)

# Get the x cols for dnn network, forecast cols
X_val_lstm = X_val.filter(regex='lag')
X_val_dnn = X_val.drop(columns=X_val_lstm.columns)

# Reorganize the training and testing data into batches
X_train_dnn, y_train_dnn = resample((X_train_dnn, y_train), 24, 24, 24)
X_val_dnn, y_val_dnn = resample((X_val_dnn, y_val), 24, 24, 24)

# LSTM
X_train_lstm, y_train_lstm = resample((X_train_lstm, y_train), 24, 24, 24)
X_val_lstm, y_val_lstm = resample((X_val_lstm, y_val), 24, 24, 24)

# Instantiate, compiled and fit dnn
input_shape = (X_train_dnn.shape[1], X_train_dnn.shape[2])
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))
dnn = compile_fit(nn, (X_train_dnn, y_train_dnn), (X_val_dnn, y_val_dnn))


# Instantiate, compiled and fit lstm
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train_lstm.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))
lstm = compile_fit(nn, (X_train_lstm, y_train_lstm), (X_val_lstm, y_val_lstm))

# Create ensemble to combine dnn and lstm, compile and fit
LSTM_DNN = ensemble_nn([dnn, lstm])
nn6 = compile_fit(LSTM_DNN, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))

# Compute metrics, add to table
results_actual['nn6'] = compute_metrics(nn6,
                                        'dnn-lstm, 1-day input',
                                        ([X_train_dnn, X_train_lstm],y_train),
                                        ([X_val_dnn, X_val_lstm],y_val))

# Save Model
nn6.save('../models/nn6')

results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,24-to-24,4.022,3.848,0.983,0.979


Looking at the table above, the combined dnn-lstm model did not improve upon the best neural network. In my [notebook](https://github.com/EvanHolder/capstone/blob/main/notebooks/NeuralNets.ipynb), I found out why the metric were degraded.  The LSTM network when trained on the `_lag` data, did not pick up on the any significant trends.  Instead, it found a few solid predictions and most of the time predicted one of four number instead of a nice continuous range.  See the plot below.

<img src='../images/LSTM_hist.png' style='width:500px;height=326px'>

In [15]:
results_actual.T.sort_values(by='SMAPE_val')

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
nn3,24-to-24,4.022,3.848,0.983,0.979
nn1,1-to-1,2.468,3.946,0.983,0.979
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
XGBoost,Vanilla,1.248,6.668,0.996,0.968
nn6,"dnn-lstm, 1-day input",3.944,7.55,0.96,0.972
nn4,"LSTM, 7-day input",5.202,8.699,0.923,0.879
nn5,"LSTM, 7-day input",5.493,8.821,0.904,0.862
TSO,None,16.03,16.922,0.954,0.971


The table above shows the results of each model, sorted by SMAPE_val.  The best models so far have been the two simple neural networks, followed by a shallow XGBoost model.  The worst models do not include `price_day_ahead`, which perform half as well across the board.  It's very clear that we need `price_day_ahead` in order to predict the actual price with any accuracy, but is it possible to predict the difference between `price_day_ahead` and `price_actual`? 

### Modeling Price Components<a class="anchor" id="Modeling-Price_Residual"></a>
`Price_actual` in reality, is made of 15 different price components, the most important being `price_day_ahead`. I'd like to know the data contains any other information about these price components. So to answer that question, there are two different approaches to try.

**Method 1: Model the Residual**<br>
In this approach, I'll subtract the `price_day_ahead` from `price_actual` to get the residual.  The residual price represents the portion of `price_actual` that is not reflected in `price_day_ahead`

**Methods 2: Model Individual Price Components**<br>
In this approach I'll set up separate models to predict each individual price component(except `price_day_ahead`). This should be able to tell us at a granular level which price components are predictable, if any.

### Model the Residual
First thing to do, create the residual column. Then remove all the other price columns

In [16]:
# Get price cols to predict
price_cols = df_lag.filter(regex='price').columns.to_list()

# Create price_residual
df_comp = df_lag.copy()
df_comp['price_residual'] = df_comp['price_day_ahead'] - df_comp['price_actual']

# Drop other price cols, and split data
df_comp.drop(columns = price_cols, inplace=True)
X_train, y_train, X_val, y_val = split_data(df_comp, 2020, 'price_residual')

Next, create another results dataframe to update with results from each model. Then I'll fit a Lasso, XGBoost, and Neural Network to the data.

In [19]:
# Create results table
results_resid = pd.DataFrame(index=['Parameters','sMAPE_train', 'sMAPE_val', 'r2_train', 'r2_val'])

# Instantiate Lasso, XGBoost
lasso_resid = Lasso(max_iter=10000)
xg_resid = XGBRegressor(random_state=17)


# Fit Lasso, XGboost
lasso_resid.fit(X_train, y_train)
xg_resid.fit(X_train, y_train)


# Compute sMAPE, r2 and add to the table
results_resid['lasso_resid']= compute_metrics(lasso_resid, 'vanilla', (X_train, y_train), (X_val, y_val))
results_resid['xg_resid'] = compute_metrics(xg_resid, 'vanilla', (X_train, y_train), (X_val, y_val))
results_resid.T

,Parameters,sMAPE_train,sMAPE_val,r2_train,r2_val
lasso_resid,vanilla,21.03,34.483,0.52,0.117
xg_resid,vanilla,9.025,35.37,0.927,0.174


As shown above, neither Lasso nor XGBoost seemed to fit the residual very well. Below I'll try a simple neural network.

In [22]:
# Define input_shape
input_shape = (X_train.shape[1],)

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(1, activation='relu'))

# Compile and Fit
nn_resid = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

# Compute metrics, add to table
results_resid['nn_resid'] = compute_metrics(nn2, 'vanilla', (X_train,y_train), (X_val, y_val))

Epoch 1/200
1369/1369 [==============================] - 8s 5ms/step - loss: 8.7495 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000
Epoch 2/200
1369/1369 [==============================] - 7s 5ms/step - loss: 7.3482 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000
Epoch 3/200
1369/1369 [==============================] - 7s 5ms/step - loss: 7.3482 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000
Epoch 4/200
1369/1369 [==============================] - 7s 5ms/step - loss: 7.3482 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000
Epoch 5/200
1369/1369 [==============================] - 7s 5ms/step - loss: 7.3482 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000
Epoch 6/200
1369/1369 [==============================] - 7s 5ms/step - loss: 7.3481 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000
Epoch 7/200
1369/1369 [==============================] - 7s 5ms/step - loss: 7.3482 - SMAPE: 200.0000 - val_loss: 5.1509 - val_SMAPE: 200.0000

In [24]:
results_resid.T

,Parameters,sMAPE_train,sMAPE_val,r2_train,r2_val
lasso_resid,vanilla,21.03,34.483,0.52,0.117
xg_resid,vanilla,9.025,35.37,0.927,0.174
nn_resid,vanilla,200.0,200.0,0.092,0.002


Again, neural network did not work either. It does not seem like there is much information contained in the residual.  Next let's try Method 2.

### Model Individual Price Components
In this method, I'll model individual price components by using sklearn's MultiOutputRegressor wrapper.  This wrapper takes in a model object, and then fits the model to training data and each given target separately.  For simplicity, I'll start with Lasso.

In [32]:
# Drop unnecessary cols
df_comps = df_lag.drop(columns=['price_day_ahead', 'price_actual'])

# Get target cols
price_cols = df_comps.filter(regex='price').columns.to_list()

# Split Data
X_train, y_train, X_val, y_val = split_data(df_comps, 2020, price_cols)

# Instantiate Lasso and Multioutput Regressor
lasso = Lasso(max_iter=10000)
l_multi = MultiOutputRegressor(lasso)

# Fit the Regressor
l_multi.fit(X_train, y_train)

# Get the predictions
preds_train = l_multi.predict(X_train)
preds_val = l_multi.predict(X_val)

# Create results table
results_lasso = pd.DataFrame(index=['Parameters','sMAPE_train', 'sMAPE_val', 'r2_train', 'r2_val'])

# Add metric results to results table
for i, model in enumerate(l_multi.estimators_):    
    results_lasso[y_val.columns[i]] = compute_metrics(model,'vanilla',(X_train,y_train.iloc[:,i]), (X_val, y_val.iloc[:,i]))
results_lasso.T.sort_values(by='sMAPE_val')

C:\Users\holde\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent(
C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\holde\flatiron\capstone\scripts\functions.py:327: RuntimeWarning: invalid value encountered in tru

,Parameters,sMAPE_train,sMAPE_val,r2_train,r2_val
price_power_factor,vanilla,11.594,16.076,0.38,0.009
price_capacity_payment,vanilla,34.371,36.169,0.628,0.645
price_PBF_tech,vanilla,47.029,38.659,0.549,0.431
price_sec_reserve,vanilla,44.785,58.79,0.463,0.413
price_measured_imbalances,vanilla,90.578,85.028,0.026,0.067
price_imbalances_net,vanilla,126.29,117.527,0.03,0.017
price_balance_failure,vanilla,132.611,127.992,0.022,0.005
price_intraday_market,vanilla,153.808,140.104,0.026,0.014
price_P0146_balance,vanilla,173.384,167.904,0.008,0.005
price_rt_tech,vanilla,162.144,172.66,0.039,0.01


XGBoost

In [34]:
# Instantiate Lasso and Multioutput Regressor
xg = XGBRegressor(random_state=17)
xg_multi = MultiOutputRegressor(xg)

# Fit the Regressor
xg_multi.fit(X_train, y_train)

# Get the predictions
preds_train = xg_multi.predict(X_train)
preds_val = xg_multi.predict(X_val)

# Create results table
results_xg = pd.DataFrame(index=['Parameters','sMAPE_train', 'sMAPE_val', 'r2_train', 'r2_val'])

# Add metric results to results table
for i, model in enumerate(xg_multi.estimators_):    
    results_xg[y_val.columns[i]] = compute_metrics(model,'Vanilla',(X_train,y_train.iloc[:,i]), (X_val, y_val.iloc[:,i]))
results_xg.T.sort_values(by='sMAPE_val')

C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,Parameters,sMAPE_train,sMAPE_val,r2_train,r2_val
price_power_factor,Vanilla,2.435,16.294,0.971,0.13
price_capacity_payment,Vanilla,11.405,38.368,0.957,0.573
price_PBF_tech,Vanilla,22.813,40.882,0.933,0.471
price_sec_reserve,Vanilla,25.511,58.703,0.874,0.297
price_measured_imbalances,Vanilla,74.216,94.997,0.844,0.002
price_imbalances_net,Vanilla,107.658,128.05,0.655,0.001
price_balance_failure,Vanilla,127.368,139.903,0.646,0.001
price_intraday_market,Vanilla,115.23,142.531,0.675,0.006
price_P0146_balance,Vanilla,123.289,148.704,0.514,0.002
price_rt_tech,Vanilla,155.139,172.34,0.743,0.016


Both the lasso and the XGBoost models have the same top 4 "modelable" components.  Lets use these models to predict their respective top four components.  The rest we'll leave out.  From there we'll use the data to predict these components, sum them together with the `price_day_ahead` and then compute their metrics and add to the original table.

In [52]:
df_comps.columns

Index(['temp_madrid_lag', 'dew_point_madrid_lag', 'humidities_madrid_lag',
       'wind_madrid_lag', 'wind_speeds_madrid_lag', 'pressures_madrid_lag',
       'condition_madrid_lag', 'temp_seville_lag', 'dew_point_seville_lag',
       'humidities_seville_lag', 'wind_seville_lag', 'wind_speeds_seville_lag',
       'pressures_seville_lag', 'condition_seville_lag', 'temp_barcelona_lag',
       'dew_point_barcelona_lag', 'humidities_barcelona_lag',
       'wind_barcelona_lag', 'wind_speeds_barcelona_lag',
       'pressures_barcelona_lag', 'condition_barcelona_lag', 'temp_bilbao_lag',
       'dew_point_bilbao_lag', 'humidities_bilbao_lag', 'wind_bilbao_lag',
       'wind_speeds_bilbao_lag', 'pressures_bilbao_lag',
       'condition_bilbao_lag', 'temp_valencia_lag', 'dew_point_valencia_lag',
       'humidities_valencia_lag', 'wind_valencia_lag',
       'wind_speeds_valencia_lag', 'pressures_valencia_lag',
       'condition_valencia_lag', 'transmission_ps_lag', 'transmission_sp_lag',
       't

In [51]:
# Components to drop
price_drop = results_xg.T.sort_values(by='sMAPE_val').iloc[4:].index.to_list().append('price_power_factor')
df_comps2 = df_comps.drop(columns=price_drop)

# Components to model
top_comps = ['price_capacity_payment', 'price_PBF_tech', 'price_sec_reserve']

# Split Data
X_train, y_train, X_val, y_val = split_data(df_comps2, 2020, top_comps)

# Instantiate Lasso and Multioutput Regressor
lasso = Lasso(max_iter=10000)
l_multi = MultiOutputRegressor(lasso)
xg = XGBRegressor(random_state=17)
xg_multi = MultiOutputRegressor(xg)

# Fit the Regressor
l_multi.fit(X_train, y_train)
xg_multi.fit(X_train, y_train)

# Get the predictions
preds_train_l = l_multi.predict(X_train).sum(axis=1) + df_lag.loc[:'2019', 'price_day_ahead']
preds_val_l = l_multi.predict(X_val).sum(axis=1) + df_lag.loc['2020', 'price_day_ahead']
preds_train_xg = xg_multi.predict(X_train).sum(axis=1) + df_lag.loc[:'2019', 'price_day_ahead']
preds_val_xg = xg_multi.predict(X_val).sum(axis=1) + df_lag.loc['2020', 'price_day_ahead']

# Get the training and validation actual price
y_true_train = df_lag.loc[:'2019','price_actual']
y_true_val = df_lag.loc['2020','price_actual']

# Add to results dataframe
results_actual['lasso3'] = ['Lasso MOR',
                            sMAPE(y_true_train, preds_train_l),
                            sMAPE(y_true_val, preds_val_l),
                            r2(y_true_train,preds_train_l),
                            r2(y_true_val,preds_val_l)]
results_actual['XGBoost4'] = ['XGBoost MOR',
                            sMAPE(y_true_train, preds_train_xg),
                            sMAPE(y_true_val, preds_val_xg),
                            r2(y_true_train,preds_train_xg),
                            r2(y_true_val,preds_val_xg)]

ValueError: Need to specify at least one of 'labels', 'index' or 'columns'

In [48]:
l_multi.predict(X_train).sum(axis=1)+df_lag.loc[:'2019', 'price_day_ahead']

2015-01-02 00:00:00    53.603231
2015-01-02 01:00:00    46.547361
2015-01-02 02:00:00    42.194146
2015-01-02 03:00:00    43.801014
2015-01-02 04:00:00    42.690630
                         ...    
2019-12-31 19:00:00    50.484978
2019-12-31 20:00:00    47.141923
2019-12-31 21:00:00    43.894074
2019-12-31 22:00:00    42.787674
2019-12-31 23:00:00    41.216047
Name: price_day_ahead, Length: 43800, dtype: float64

In [45]:
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.021,5.869,0.977,0.973
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn1,1-to-1,2.468,3.946,0.983,0.979
nn2,"1-to-1, price_day_ahead:False",6.849,23.042,0.907,0.428


## Results
___

## Conclusion
___

## Next Steps
___